In [49]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [8]:
df = pd.read_csv("./wrangled/manufacturing imputed.csv")
df = df.pivot_table(index='year', columns='type', values='value')
df = df[(df.index >= 2000) & (df.index <= 2017)]
pol = pd.read_csv("./wrangled/pollution.csv")
pol = pol[(pol["year"] >= 2000) & (pol['year'] <= 2017)]
pol

,year,sulphur_dioxide_mean,co_max_8hour_mean,ozone_maximum_8hour_mean,nitrogen_dioxide_mean
0,2000,22,3.7,112,30
1,2001,22,4.2,133,26
2,2002,18,2.7,131,27
3,2003,15,3.2,118,24
4,2004,14,2.8,146,26
5,2005,14,2.4,159,25
6,2006,11,2.6,136,24
7,2007,12,1.7,206,22
8,2008,11,1.6,183,22
9,2009,9,1.9,105,22


In [4]:
df

level_2,Basic Metal,Chemicals & Chemical Products,"Computer, Electronic & Optical Products",Electrical Equipment,Fabricated Metal Products,"Food, Beverage & Tobacco",Furniture,"Leather, Leather Products & Footwear",Machinery & Equipment,"Motor Vehicles, Trailers & Semi-Trailers",...,Other Manufacturing Industries,Other Transport Equipment,Paper & Paper Products,Pharmaceutical & Biological Products,Printing & Reproduction Of Recorded Media,Refined Petroleum Products,Rubber & Plastic Products,Textiles & Textile Manufactures,Wearing Apparel,Wood & Wood Products
year,,,,,,,,,,,,,,,,,,,,,
2000,503.9,11682.1,86297.8,2559.9,7510.3,3750.9,663.5,208.6,7610.1,240.2,...,1769.1,5872.2,876.8,4841.5,3078.3,20096.5,3461.3,275.8,1094.2,293.5
2001,509.0,11735.6,64880.2,2172.8,6342.9,3821.9,600.6,192.9,7047.8,195.7,...,1912.6,7224.5,816.7,5119.2,2782.0,18271.7,2785.2,262.5,859.7,255.4
2002,570.7,14292.9,64663.4,2049.1,6692.9,4357.2,725.1,191.4,8059.4,209.7,...,2347.5,8114.6,880.1,8063.7,2666.5,18419.6,2980.1,225.6,882.6,268.3
2003,510.5,18354.3,66338.4,1984.5,6656.0,4286.2,665.8,208.6,8226.6,896.0,...,2379.5,7660.9,891.5,10089.0,2722.9,21981.3,2872.4,250.7,859.2,260.1
2004,764.0,24265.5,76839.9,2426.3,7435.6,5135.9,629.8,213.5,9881.2,827.0,...,2115.5,8956.0,927.7,15380.9,2737.3,28275.3,2857.8,229.7,881.0,282.9
2005,1023.2,27058.3,82243.0,2570.9,7734.6,5517.4,598.6,216.5,11850.3,796.8,...,2383.1,10742.9,1013.4,15865.3,2836.8,40307.2,2859.9,213.5,684.5,269.4
2006,1234.4,32437.9,79316.1,2953.0,8742.5,5760.1,564.4,239.4,14270.1,847.5,...,2424.6,13633.6,1003.8,21093.0,2974.3,46118.4,2965.1,168.6,586.7,279.3
2007,1368.6,35457.3,82247.4,3161.4,9176.5,6286.7,621.3,247.3,19028.6,795.0,...,3293.5,14271.7,1053.3,20838.3,3028.3,48047.0,2678.3,143.2,730.1,258.8
2008,1705.8,35044.4,78352.5,3298.0,9831.9,7692.8,837.0,212.5,20461.1,650.8,...,3408.4,16481.0,1100.1,17168.8,3094.4,59950.8,2518.5,145.1,565.2,290.9


In [41]:
results = pd.DataFrame(columns=["manufactured","pollutant","r2","count","mse"])
manuCategories = [["Food, Beverage & Tobacco","Other Manufacturing Industries"],
                  ["Textiles & Textile Manufactures","Wearing Apparel","Leather, Leather Products & Footwear","Rubber & Plastic Products"],
                  ["Wood & Wood Products","Paper & Paper Products","Printing & Reproduction Of Recorded Media","Furniture"],
                  ["Chemicals & Chemical Products","Refined Petroleum Products","Pharmaceutical & Biological Products"],
                  ["Non-Metallic Mineral Products","Basic Metal","Fabricated Metal Products","Computer, Electronic & Optical Products","Electrical Equipment","Machinery & Equipment","Motor Vehicles, Trailers & Semi-Trailers","Other Transport Equipment"]]
pollution = ["sulphur_dioxide_mean","co_max_8hour_mean"
             ,"ozone_maximum_8hour_mean","nitrogen_dioxide_mean"]


In [48]:
x =0
for l in range(0,5):
    for i in pollution:
        for k in manuCategories:
            X = df[k]
            y = pol[i]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

            regressor = LinearRegression()
            regressor.fit(X_train, y_train)
            
            test = regressor.score(X_test,y_test)
            y_pred = regressor.predict(X_test)
            mse = metrics.mean_squared_error(y_test, y_pred)
            temp = pd.DataFrame({"manufactured":"".join(k),"pollutant":i,"r2":test,"count":l,"mse":mse},index=[x])
            results = pd.concat([results,temp])
            x = x+1
results = results.groupby(['manufactured', 'pollutant'])[['r2',"mse"]].mean()
pd.DataFrame(results)


r2  \
manufactured                                       pollutant                              
Chemicals & Chemical ProductsRefined Petroleum ... co_max_8hour_mean          -4.100402   
                                                   nitrogen_dioxide_mean      -1.533047   
                                                   ozone_maximum_8hour_mean   -2.946272   
                                                   sulphur_dioxide_mean        0.522380   
Food, Beverage & TobaccoOther Manufacturing Ind... co_max_8hour_mean          -2.404433   
                                                   nitrogen_dioxide_mean      -2.316913   
                                                   ozone_maximum_8hour_mean   -6.125383   
                                                   sulphur_dioxide_mean       -2.031570   
Non-Metallic Mineral ProductsBasic MetalFabrica... co_max_8hour_mean         -52.440067   
                                                   nitrogen_dioxide_mean      -1.937124   
                                                   ozone_maximum_8hour_mean -130.390731   
                                                   sulphur_dioxide_mean      -25.785785   
Textiles & Textile ManufacturesWearing ApparelL... co_max_8hour_mean        -157.157204   
                                                   nitrogen_dioxide_mean      -2.302742   
                                                   ozone_maximum_8hour_mean   -3.422399   
                                                   sulphur_dioxide_mean       -9.169325   
Wood & Wood ProductsPaper & Paper ProductsPrint... co_max_8hour_mean          -4.933281   
                                                   nitrogen_dioxide_mean      -8.045873   
                                                   ozone_maximum_8hour_mean   -5.240623   
                                                   sulphur_dioxide_mean       -1.174710   

                                                                                     mse  
manufactured                                       pollutant                              
Chemicals & Chemical ProductsRefined Petroleum ... co_max_8hour_mean            0.992312  
                                                   nitrogen_dioxide_mean        5.085112  
                                                   ozone_maximum_8hour_mean   723.762762  
                                                   sulphur_dioxide_mean         4.544346  
Food, Beverage & TobaccoOther Manufacturing Ind... co_max_8hour_mean            0.811923  
                                                   nitrogen_dioxide_mean        8.946611  
                                                   ozone_maximum_8hour_mean   449.758371  
                                                   sulphur_dioxide_mean         6.921766  
Non-Metallic Mineral ProductsBasic MetalFabrica... co_max_8hour_mean            0.808293  
                                                   nitrogen_dioxide_mean       13.354585  
                                                   ozone_maximum_8hour_mean  3612.713991  
                                                   sulphur_dioxide_mean        17.196403  
Textiles & Textile ManufacturesWearing ApparelL... co_max_8hour_mean            2.126301  
                                                   nitrogen_dioxide_mean        5.069410  
                                                   ozone_maximum_8hour_mean   894.201903  
                                                   sulphur_dioxide_mean         9.236049  
Wood & Wood ProductsPaper & Paper ProductsPrint... co_max_8hour_mean            1.775852  
                                                   nitrogen_dioxide_mean        2.609300  
                                                   ozone_maximum_8hour_mean  1045.911661  
                                                   sulphur_dioxide_mean         3.686496

In [7]:
results.to_csv("./wrangled/regression2222.csv")



